# Practical Guide

This guide will help you with all the practical aspects of setting up an annotation project for training and fine-tuning LLMs using Argilla's Feedback Task Datasets. It covers everything from defining your task to collecting, organizing, and using the feedback effectively.

[Add a snapshot of the task]

In [1]:
import argilla as rg
import os

rg.init(
    api_url=os.environ.get("ARGILLA_API_URL_DEV"),
    api_key=os.environ.get("ARGILLA_API_KEY")
)

## Define the task
The Feedback Task Datasets allow to combine multiple questions of different kinds, so the first step will be to define the aim of your project and the kind of data and feedback you will need to get there.

### Format records
A record in Argilla refers to a data item that requires annotation and can consist of one or multiple fields. For example, your records can include a pair of a prompt and an output. Currently, we only support plain text fields, but we plan to introduce support for markdown and images in the future.

Take some time to explore and find data that fits the purpose of your project. If you are planning to use public data, the [Datasets page](https://huggingface.co/datasets) of the Hugging Face Hub is a good place to start.

´´´{hint}
Always check the licenses of the datasets to make sure you can legally use the dataset for your specfic use case.
´´´

Once you have a dataset, load it and inspect it to find the fields that you want to use in your Feedback dataset. A quick overview of the data will also help you formulate the right questions later.

In [2]:
from datasets import load_dataset

dataset = load_dataset('databricks/databricks-dolly-15k', split='train')
dataset

/Users/natalia/opt/anaconda3/envs/argilla/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[05/18/23 11:10:51] WARNING  WARNING:datasets.builder:Found cached dataset json                      ]8;id=417076;file:///Users/natalia/opt/anaconda3/envs/argilla/lib/python3.9/site-packages/datasets/builder.py\builder.py]8;;\:]8;id=916231;file:///Users/natalia/opt/anaconda3/envs/argilla/lib/python3.9/site-packages/datasets/builder.py#816\816]8;;\
                             (/Users/natalia/.cache/huggingface/datasets/databricks___json/databrick               
                             s--databricks-dolly-15k-6e0f9ea7eaa0ee08/0.0.0/e347ab1c932092252e717ff3               
                             f949105a4dd28b27e842dd53157d2f72e276c2e4)                                             

Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 15011
})

In [ ]:
import pandas as pd

# turn it into a pandas dataframe to get a quick overview of a few examples
df = pd.DataFrame(dataset)
df

The next step is to create records following Argilla's Feedback Record format [link to Python reference].

The name of the fields will need to match the fields set up in the dataset configuration (see [below](#create-your-dataset)).

In [3]:
# list of records
records = []
for r in dataset:
    if r["category"] == "open_qa":
        records.append(
            rg.FeedbackRecord(
                fields = {
                    "question": r["instruction"],
                    "answer": r["response"]
                }
            )
        )


### Define questions
To collect feedback for your dataset, you need to formulate questions. The Feedback Task currently supports the following types of questions:

- Rating: These questions require annotators to select one option from a list of integer values. This type is useful for collecting numerical scores.
- Text: These questions offer annotators a free-text area where they can enter any text. This type is useful for collecting natural language data, such as corrections or explanations.

```{note}
We have plans to expand the range of supported question types in future releases of the Feedback Task.
```

You can define your questions using the Python SDK and set up the following configurations:
- `name`: A shortname for the question.
- `title`: The text displayed in the UI.
- `description`: The text to be displayed in the question tooltip in the UI. You can use it to give more context or information to annotators.
- `required`: Set your question as required or optional. Annotators must answer all required questions to submit a response, but they have the choice to answer optional questions or not.
- `values`: In a RatingQuestion, these are the rating options represented as a list of integer values.

```{note}
The order of the questions in the UI follows the order in which these are added to the dataset in the Python SDK.
```

In [4]:
# list of questions to display in the feedback form
questions =[
    rg.RatingQuestion(
        name="rating", 
        title="Rate the quality of the response:", 
        description="1 = very bad - 5= very good",
        required=True,
        values=[1,2,3,4,5]
    ),
    rg.TextQuestion(
        name="corrected-text",
        title="Provide a correction to the response:",
        description="",
        required=False
    )
]


### Write guidelines
Once you have decided on the data to show and the questions to ask, it's important to provide clear guidelines to the annotators. These guidelines help them understand the task and answer the questions consistently. You can provide guidelines in two ways:
- In the dataset guidelines: this is added as an argument when you create your dataset in the Python SDK (see below). It will appear in the dataset settings in the UI.
- As question descriptions: these are added as an argument when you create questions in the Python SDK (see above). This text will appear in a tooltip next to the question in the UI.

It is good practice to use at least the dataset guidelines, if not both methods. In the guidelines, you can include a description of the project, details on how to answer each question with examples, instructions on when to discard a record, etc. Question descriptions should be short and provide context to a specific question. They can be a summary of the guidelines to that question, but often times that is not sufficient to align the whole annotation team.

## Set up your annotation team
Depending on the nature of your project and the size of your annotation team, you may want to have control over annotation overlap i.e., having multiple annotations for a single record. You will need to decide on this before pushing your dataset to Argilla, as this has implications on how your dataset is set up. Let's explore a few overlapping options.

### Full overlap
The Feedback Task supports having multiple annotations for your records. This means that all users with access to the dataset can give responses to all the records in the dataset. To have this full overlap just push the dataset (as detailed in [Create your dataset](#create-your-dataset)) in a workspace where all team members have access. Learn more about managing user access to workspaces [here](../getting_started/installation/configurations/user_management.md#creating-an-annotator-user-assigned-to-a-workspace).

### Zero overlap
If you only want one annotation per record, we recommend that you split your records into chunks and assign these to a single annotator. Then, you can create several datasets, one in each annotator's personal workspace, and add the records assigned to each of them.

In [ ]:

import httpx
import random
from collections import defaultdict

# make a request using your Argilla Client
rg_client= rg.active_client().client
auth_headers = {"X-Argilla-API-Key": rg_client.token}
http=httpx.Client(base_url=rg_client.base_url, headers=auth_headers)
users = http.get("/api/users").json()

# optional: filter users to get only those with annotator role
users = [u for u in users if u['role']=='annotator']

# optional: shuffle the records to get a random assignment
random.shuffle(records)

# build a dictionary where the key is the username and the value is the list of records assigned to them
assignments = defaultdict(list)

# divide your records in chunks of the same length as the users list and make the assignments
# you will need to follow the instructions to create and push a dataset for each of the key-value pairs in this dictionary
n = len(users)
chunked_records = [records[i:i + n] for i in range(0, len(records), n)]
for chunk in chunked_records:
    for idx, record in enumerate(chunk):
        assignments[users[idx]['username']].append(record)

### Controlled overlap
This option is optimal when you want to have annotation overlap, but up to a certain number and not with the whole team. This can be because you want your team to be more efficient or perhaps to calculate the agreement between pairs of annotators. In this case, you also need to create several datasets and push them to the annotators' personal workspaces with the difference that each record will appear in multiple datasets. 

In [ ]:
# code to assign with overlap

## Create your dataset
Now we are ready to create our dataset. To do that, first you'll need to define the following configurations:
- `name`: The name of the dataset.
- `workspace`: The workspace where the dataset will be created.
- `guidelines`: A set of guidelines for the annotators. These will appear in the dataset settings in the UI. If you want this to be empty, leave it as an empty string.
- `fields`: The list of fields to show in the record card. The order in which the fields will appear matches the order of this list.
- `questions`: The list of questions to show in the form.

In [5]:
# configure the Feedback dataset
dataset = rg.create_feedback_dataset(
    name="demo_feedback",
    workspace="recognai",
    guidelines="You will see a collection of records with a question and an answer. You will be asked to rate the answer from 1 (very bad) to 5 (very good) and, if needed, provide a correction to the output",
    fields = [
        rg.TextField(name="question"),
        rg.TextField(name="answer")
    ],
    questions=questions
)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /Users/natalia/opt/anaconda3/envs/argilla/lib/python3.9/site-packages/argilla/client/feedback.py │
│ :547 in push_to_argilla                                                                          │
│                                                                                                  │
│   544 │   │   │                                                                                  │
│   545 │   │   │   for question in self.questions:                                                │
│   546 │   │   │   │   try:                                                                       │
│ ❱ 547 │   │   │   │   │   client.add_question(id=argilla_id, question=question.dict())           │
│   548 │   │   │   │   except Exception as e:                                                     │
│   549 │   │   │   │   │   delete_and_raise_exception(                                            │
│   550 │   │   │   │   │   │   dataset_id=argilla_id,                                             │
│                                                                                                  │
│ /Users/natalia/opt/anaconda3/envs/argilla/lib/python3.9/site-packages/argilla/client/client.py:2 │
│ 62 in add_question                                                                               │
│                                                                                                  │
│   259 │   │   return datasets_api_v1.get_questions(client=self._client, id=id).parsed            │
│   260 │                                                                                          │
│   261 │   def add_question(self, id: str, question: Dict[str, Any]) -> httpx.Response:           │
│ ❱ 262 │   │   return datasets_api_v1.add_question(client=self._client, id=id, question=questio   │
│   263 │                                                                                          │
│   264 │   def copy(self, dataset: str, name_of_copy: str, workspace: str = None):                │
│   265 │   │   """Creates a copy of a dataset including its tags and metadata                     │
│                                                                                                  │
│ /Users/natalia/opt/anaconda3/envs/argilla/lib/python3.9/site-packages/argilla/client/sdk/v1/data │
│ sets/api.py:303 in add_question                                                                  │
│                                                                                                  │
│   300 │   │   │   content=response.content,                                                      │
│   301 │   │   │   headers=response.headers,                                                      │
│   302 │   │   )                                                                                  │
│ ❱ 303 │   return handle_response_error(response)                                                 │
│   304                                                                                            │
│                                                                                                  │
│ /Users/natalia/opt/anaconda3/envs/argilla/lib/python3.9/site-packages/argilla/client/sdk/commons │
│ /errors_handler.py:64 in handle_response_error                                                   │
│                                                                                                  │
│   61 │   │   error_type = GenericApiError                                                        │
│   62 │   else:                                                                                   │
│   63 │   │   raise HttpResponseError(response=response)                                          │
│ ❱ 64 │   raise error_type(**error_args)                                                          │
│   65                                                       

Once the dataset is created, add the records and, when you're happy with the result, push the dataset to Argilla. At that point, you will be able to see the dataset from the UI.

In [ ]:
# add the records to the dataset
dataset.add_records(records)

# push the dataset to Argilla
dataset.push_to_argilla()

## Annotating a Feedback Dataset
Once you open the dataset, you will see by default the records with `Pending` responses, i.e. records that still don't have a response, in a single-record view. On the left, you can find the record to annotate and on the right the form with all the questions to answer. 

We highly recommend that you read the annotation guidelines before starting the annotation. If there are any, you can find them in the dataset settings page. [describe how to get there] If any of the questions have a description, you will find an info icon next to them. Click it to read the description.

In the annotation view, you will be able to provide responses. Once all required questions have responses the `Submit` button will be enabled and you will be able to submit your response. If you prefer not to give a response for a record, you can move to the next record or discard it using the `Discard` button. 

If you need to review your submitted or discarded responses, you can select the view/queue? you need. From there, you can modify, submit or discard responses. You can also use the `Clear` button to remove the response and send the record back to the `Pending` queue.

You can speed up the annotation process by using shortcuts:
|Action|Keys|
|------|----|
|Clear|&#8679; `Shift` + &blank; `Space`|
|Discard|&#x232B; `Backspace`|
|Discard (from text area)|&#8679; `Shift` + &#x232B; `Backspace`|
|Submit|&crarr; `Enter`|
|Submit (from text area)|&#8679; `Shift` + &crarr; `Enter`|
|Go to previous page|&larr; `Left arrow`|
|Go to next page|&rarr; `Right arrow`|

You can track your progress and the number of `Pending`, `Submitted` and `Discarded` responses by clicking the `Progress` icon in the sidebar.

## Collect responses

- Using the Python client to collect the responses to the dataset
- Unifying responses (?) -> Técnicas de IAA.
    - Majority vote, average... 
    - How to calculate IAA for text fields? bleu rouge? Rating of the proposed texts. 
    Make a dataset to collect human text, then rate the human text and use it for a rating exercise to get an annotator score or clean the dataset.
- Export / publish the dataset.

## Training?